In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from collections import defaultdict
import os, sys

mx_base = "/mnt/extproj/projekte/textmining/mxplore/"
model_base = mx_base + "spacy_models/"
mirexplore_base = mx_base + "/python/textmine/"

if not mirexplore_base in sys.path:
    sys.path.insert(0, mirexplore_base)
    
import pickle

import scispacy
import spacy
from spacy import displacy

from MirGeneRelCheck import SentenceRelationChecker, SentenceRelationClassifier
from mxutils.tmutils import normalize_gene_names
from collections import Counter
from lxml import etree
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from itertools import chain, combinations



In [3]:
spacy.__version__

'2.2.1'

In [4]:
nlp_weblg = spacy.load(model_base+"/en_core_web_lg-2.2.0/en_core_web_lg/en_core_web_lg-2.2.0/")
nlp = spacy.load(model_base+"/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/")
nlp_ent = spacy.load(model_base+"/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")


In [5]:
nlp.meta['name']

'core_sci_lg'

In [6]:
%aimport MirGeneRelCheck

scaiBase = mx_base + "/scai_eval/"

normGeneSymbols= normalize_gene_names(path=scaiBase + "hgnc_no_withdrawn.syn")

def loadRelations(relsFile):
    reldir2new = {}
    reldir2new["TARGET"] = "NEU"
    reldir2new["INVCORR"] = "NEU"
    reldir2new["REGULATE"] = "NEU"
    relClasses = {}
    with open(os.path.join(scaiBase, relsFile)) as fin:
        
        print("Loading relations", relsFile)
        
        for lidx, line in enumerate(fin):

            if lidx == 0:
                continue

            line = line.strip().split("\t")

            interactionID = line[0].strip()
            relType = line[1].strip()
            relRDir = line[2].strip()
            relIndirect = line[3].strip()
            relPassive = line[4].strip()

            if relRDir in reldir2new:
                relRDir = reldir2new[relRDir]

            relClasses[interactionID] = (relType, relRDir, relIndirect, relPassive)
            
        print("Interactions", len(relClasses))
        print("Non NA Interactions", len([x for x in relClasses if not relClasses[x][0] == "NA"]))
            
    return relClasses

def all_subsets(ss):
    
    return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

def runCheck(numelems, base, nlp, nlp_ent, relationsFile, subset_interactions=False, subset_classification=False):

    relChecker = SentenceRelationChecker(None)
    relClassifier = SentenceRelationClassifier(relationsFile)
    
    if base == "TRAIN":
        scaiFile = "miRNA_train_fixed.xml"
        relsFile = "scai_train_rels.tsv"
    elif base == "TEST":
        scaiFile = "miRNA_test_fixed.xml"
        relsFile = "scai_test_rels.tsv"
    
    relClasses = loadRelations(relsFile)
    
    check2results = {}
    checkInteractionSubsets = [x for x in all_subsets(relChecker.relCheck.checks_available)]
    checkClassifierSubsets = [x for x in all_subsets(relClassifier.major_checks)]
    #checkSubsets = [checkSubsets[-1]]
    
    if not subset_interactions:
        checkInteractionSubsets = [checkInteractionSubsets[-1]]
        
    if not subset_classification:
        checkClassifierSubsets = [checkClassifierSubsets[-1]]
    
    print("Will test interaction: {} subsets.".format(len(checkInteractionSubsets)))
    print("Will test classification: {} subsets.".format(len(checkClassifierSubsets)))
    
    for subset_checks_interaction in checkInteractionSubsets:
        for subset_checks_classification in checkClassifierSubsets:

            print("Testing checks", subset_checks_interaction)
            print("Testing checks", subset_checks_classification)
            
            relChecker = SentenceRelationChecker(nlp, nlp_ent=nlp_ent, active_checks=subset_checks_interaction)
            relClassifier = SentenceRelationClassifier(relationsFile, active_checks=subset_checks_classification)

            with open(os.path.join(scaiBase, scaiFile), 'r') as fin:
                tree = etree.parse(fin)
                root = tree.getroot()
                scaiPairs = []
                totalChecks = 0
                correctIdentified = 0
                incorrectIdentified = 0

                incorrectClassified = 0
                correctClassified = 0
                totalClassified = 0
                correctInteractClassified = 0
                totalValidClassified = 0
                totalClassifiable = 0
                totalMirGeneDownClassified = 0

                errorByDetect = Counter()
                classifiedByDetect = Counter()
                elemCaseCounter = Counter()
                incorrectClass = Counter()

                classifyTrue = []
                classifyPred = []

                for elem in root.findall(".//document"):

                    pmid = elem.attrib['origId']

                    for sentElem in elem:

                        allEntities = sentElem.findall(".//entity")
                        allPairs = sentElem.findall(".//pair")

                        sentText = sentElem.attrib["text"]

                        entId2elem = {}

                        for entity in allEntities:
                            entId = entity.attrib['id']
                            entText = entity.attrib['text']
                            entType = entity.attrib['type']
                            entOffset = tuple([int(x) for x in entity.attrib['charOffset'].split("-")])

                            if entType in ["Specific_miRNAs", "Genes/Proteins"]:

                                if "Genes" in entType:
                                    if entText in normGeneSymbols:
                                        entText = normGeneSymbols[entText]
                                    elif entText.upper() in normGeneSymbols:
                                        gene = normGeneSymbols[entText.upper()]
                                else:
                                    try:
                                        entText = miRNA(entText).getStringFromParts([miRNAPART.MATURE, miRNAPART.ID, miRNAPART.PRECURSOR])
                                    except:
                                        pass

                                entTuple = (entText, entType, (entOffset[0], entOffset[1]+1))
                                entId2elem[entId] = entTuple


                                sentEntText = sentText[entTuple[2][0]:entTuple[2][1]]

                        for pair in allPairs:

                            validInteraction = pair.attrib['interaction'].lower() == "true"
                            pairE1 = pair.attrib['e1']
                            pairE2 = pair.attrib['e2']

                            #if pairInt == 'true':
                            if pairE1 in entId2elem and pairE2 in entId2elem:

                                totalChecks += 1

                                e1 = entId2elem[pairE1]
                                e2 = entId2elem[pairE2]

                                if not e1[1] in ["Specific_miRNAs"]:

                                    tmp=e1
                                    e1=e2
                                    e2=tmp


                                relRes = relChecker.check_sentence(sentText
                                                                    , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                    , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                    , fix_special_chars=False
                                                                    , relClassifier=relClassifier.classify, verbose=False
                                                                    )


                                fullsentence = relRes['full_sentence']
                                acceptInteraction = relRes['accept_relation']

                                if not acceptInteraction == validInteraction:

                                    incorrectIdentified += 1

                                    """
                                    relResV=relChecker.check_sentence(sentText
                                                                    , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                    , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                    , fix_special_chars=False
                                                                    , relClassifier=relClassifier.classify, verbose=True
                                                                    )

                                    print(relResV)
                                    """
                                else:
                                    correctIdentified += 1

                                elemCaseCounter[(validInteraction, acceptInteraction)]+=1

                                totalClassified += 1

                                if validInteraction:# and acceptInteraction:
                                    #print(relRes["check_results"])   
                                    totalValidClassified += 1

                                    foundClasses = relRes["check_results"]["classification"] #{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'regulation_keys': set()}
                                    foundTuple = (foundClasses["interaction_dir"], foundClasses["regulation_dir"])                                
                                    relationID = pair.attrib["id"]
                                    testTuple = relClasses[relationID]

                                    classifyTrue.append((testTuple[0], testTuple[1]))
                                    classifyPred.append((foundTuple[0], foundTuple[1]))

                                    if testTuple[0] == "MIR_GENE" and testTuple[1] == "DOWN":
                                        totalMirGeneDownClassified += 1

                                    classifiedByDetect[foundClasses["reg_detect"]] += 1

                                    if testTuple[0] != foundTuple[0] or testTuple[1] != foundTuple[1]:


                                        #print(sentText)
                                        #print(e1, e2)
                                        #print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                        #print(testTuple)

                                        #print()
                                        #print()
                                        #print()

                                        #print(relationID)
                                        #print("IS: ", foundTuple)
                                        #print("SB: ", testTuple)
                                        #print(relRes)

                                        #relResV=relChecker.check_sentence(sentText
                                        #                            , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                        #                            , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                        #                            , fix_special_chars=False
                                        #                            , relClassifier=relClassifier.classify, verbose=True
                                        #                            )

                                        #print(relResV)

                                        #print()
                                        #print()
                                        #print()

                                        errorByDetect[foundClasses["reg_detect"]] += 1



                                        incorrectClassified += 1



                                        #numelems -= 1
                                        #if numelems == 0:
                                        #    return
                                    else:

                                        if foundClasses["reg_detect_major"] == "return":
                                            pass
                                            #print(foundClasses)
                                            #print(sentText)
                                            #print(e1, e2)
                                            #print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                            #print(testTuple)
                                            #print()
                                            #print()
                                            #print()
                                        correctClassified += 1

                                        if acceptInteraction:
                                            correctInteractClassified += 1



                def printStats(outfile):
                    print("Total:     ", totalChecks, file=outfile)
                    print("Correct:   ", correctIdentified, correctIdentified/totalChecks, file=outfile)
                    print("Incorrect: ", incorrectIdentified, incorrectIdentified/totalChecks, file=outfile)
                    print("classes", incorrectClass, file=outfile)

                    precision = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(True, False)])
                    recall = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(False, True)])

                    f1 = 2* precision * recall / (precision+recall)

                    if (elemCaseCounter[(True, False)] + elemCaseCounter[(False, False)]) == 0:
                        specificity = 0
                    else:
                        specificity = elemCaseCounter[(False, False)] / (elemCaseCounter[(True, False)] + elemCaseCounter[(False, False)])

                    print("precision", precision, file=outfile)
                    print("recall", recall, file=outfile)
                    print("specificity", specificity, file=outfile)
                    print("f1", f1, file=outfile)

                    print("Correct classified:     ", correctClassified, correctClassified/totalClassified, correctClassified/totalValidClassified,file=outfile)
                    print("Incorrect classified:   ", incorrectClassified, incorrectClassified/totalClassified, incorrectClassified/totalValidClassified,file=outfile)
                    print("Random classified: ", totalMirGeneDownClassified, totalMirGeneDownClassified / totalClassified, totalMirGeneDownClassified/totalValidClassified, file=outfile)
                    print(errorByDetect)

                    print("Correct interaction&classified:     ", correctInteractClassified, correctInteractClassified/totalClassified, correctInteractClassified/totalValidClassified,file=outfile)

                    print("Classified by: ", classifiedByDetect)

                    lClassifyTrue = ["_".join(x) for x in classifyTrue]
                    lClassifyPred = ["_".join(x) for x in classifyPred]

                    allLabels = sorted(set(lClassifyTrue+lClassifyPred))
                    matrix = multilabel_confusion_matrix(lClassifyTrue, lClassifyPred, labels=allLabels)
                    print(matrix)
                    print(classification_report(lClassifyTrue,lClassifyPred))

                    return {'interaction': {'precision': precision, 'recall': recall, 'specificity': specificity, 'f1': f1}, 'regulation': classification_report(lClassifyTrue, lClassifyPred, output_dict=True) }

                rep = printStats(sys.stdout)
                check2results[(subset_checks_interaction, subset_checks_classification)] = rep
        #printStats(sys.stderr)
    return check2results
        


In [7]:
relationsFile = mx_base + "/obodir/allrels.csv"

In [8]:
%aimport MirGeneRelCheck

results_test_bionlp_bionlp_subseti = runCheck(-1, "TEST", nlp_ent, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)
results_train_bionlp_bionlp_subseti = runCheck(-1, "TRAIN", nlp_ent, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 32 subsets.
Will test classification: 1 subsets.
Testing checks ()
Testing checks ('compartment', 'between', 'counts', 'return')
Total:      232
Correct:    139 0.5991379310344828
Incorrect:  93 0.40086206896551724
classes Counter()
precision 1.0
recall 0.5991379310344828
specificity 0
f1 0.7493261455525607
Correct classified:      128 0.5517241379310345 0.920863309352518
Incorrect classified:    11 0.04741379310344827 0.07913669064748201
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1})
Correct interaction&classified:      128 0.5517241379310345 0.920863309352518
Classified by:  Counter({'counts between': 66, 'counts between equal': 10, 'between gene mir': 10, 'counts opp': 9, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'between mir gene reg co

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    145 0.625
Incorrect:  87 0.375
classes Counter()
precision 0.9136690647482014
recall 0.6287128712871287
specificity 0.6
f1 0.7448680351906158
Correct classified:      128 0.5517241379310345 0.920863309352518
Incorrect classified:    11 0.04741379310344827 0.07913669064748201
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1})
Correct interaction&classified:      116 0.5 0.8345323741007195
Classified by:  Counter({'counts between': 66, 'counts between equal': 10, 'between gene mir': 10, 'counts opp': 9, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'between mir gene reg corr': 5, 'compartment mir gene neg corr': 4, 'compartment gene mir by': 2, 'counts between alternating': 1})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[ 81   4]
  [  5  49]]

 [[ 57   3]
  [  6  73]]

 [[136   2]
  [  0   1]]]
               p

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


...

Total:      397
Correct:    330 0.8312342569269522
Incorrect:  67 0.16876574307304787
classes Counter()
precision 0.8814432989690721
recall 0.7953488372093023
specificity 0.8736263736263736
f1 0.8361858190709046
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      150 0.3778337531486146 0.7731958762886598
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    320 0.8060453400503779
Incorrect:  77 0.19395465994962216
classes Counter()
precision 0.9020618556701031
recall 0.7510729613733905
specificity 0.8841463414634146
f1 0.819672131147541
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      153 0.3853904282115869 0.788659793814433
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene p

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    332 0.836272040302267
Incorrect:  65 0.163727959697733
classes Counter()
precision 0.8814432989690721
recall 0.8028169014084507
specificity 0.875
f1 0.8402948402948403
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      150 0.3778337531486146 0.7731958762886598
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene pos corr': 4, '

In [9]:
%aimport MirGeneRelCheck

results_test_scilg_bionlp_subseti = runCheck(-1, "TEST", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)
results_train_scilg_bionlp_subseti = runCheck(-1, "TRAIN", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 32 subsets.
Will test classification: 1 subsets.
Testing checks ()
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    139 0.5991379310344828
Incorrect:  93 0.40086206896551724
classes Counter()
precision 1.0
recall 0.5991379310344828
specificity 0
f1 0.7493261455525607
Correct classified:      127 0.5474137931034483 0.9136690647482014
Incorrect classified:    12 0.05172413793103448 0.08633093525179857
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      127 0.5474137931034483 0.9136690647482014
Classified by:  Counter({'counts between': 64, 'counts opp': 10, 'counts between equal': 10, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'compartment gene mir by': 4, 'compartment mir gene neg corr': 4, 'between mir gene reg corr': 3, 'counts between alternating': 2})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[ 80   5]
  [  5  49]]

 [[ 57  

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    159 0.6853448275862069
Incorrect:  73 0.3146551724137931
classes Counter()
precision 0.9424460431654677
recall 0.6683673469387755
specificity 0.7777777777777778
f1 0.7820895522388059
Correct classified:      127 0.5474137931034483 0.9136690647482014
Incorrect classified:    12 0.05172413793103448 0.08633093525179857
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      120 0.5172413793103449 0.8633093525179856
Classified by:  Counter({'counts between': 64, 'counts opp': 10, 'counts between equal': 10, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'compartment gene mir by': 4, 'compartment mir gene neg corr': 4, 'between mir gene reg corr': 3, 'counts between alternating': 2})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[

...

Total:      397
Correct:    354 0.8916876574307305
Incorrect:  43 0.10831234256926953
classes Counter()
precision 0.9278350515463918
recall 0.861244019138756
specificity 0.925531914893617
f1 0.8933002481389579
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      156 0.3929471032745592 0.8041237113402062
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene p

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    343 0.8639798488664987
Incorrect:  54 0.13602015113350127
classes Counter()
precision 0.9536082474226805
recall 0.8043478260869565
specificity 0.9461077844311377
f1 0.8726415094339622
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      160 0.40302267002518893 0.8247422680412371
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gen

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    359 0.9042821158690176
Incorrect:  38 0.09571788413098237
classes Counter()
precision 0.9278350515463918
recall 0.8823529411764706
specificity 0.927461139896373
f1 0.9045226130653266
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      156 0.3929471032745592 0.8041237113402062
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene 

In [10]:
%aimport MirGeneRelCheck

results_test_spcylg_bionlp_subseti = runCheck(-1, "TEST", nlp_weblg, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)
results_train_spcylg_bionlp_subseti = runCheck(-1, "TRAIN", nlp_weblg, nlp_ent, relationsFile=relationsFile, subset_interactions=True, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 32 subsets.
Will test classification: 1 subsets.
Testing checks ()
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    139 0.5991379310344828
Incorrect:  93 0.40086206896551724
classes Counter()
precision 1.0
recall 0.5991379310344828
specificity 0
f1 0.7493261455525607
Correct classified:      111 0.47844827586206895 0.7985611510791367
Incorrect classified:    28 0.1206896551724138 0.2014388489208633
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 24, 'between mir gene': 2, 'compartment mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      111 0.47844827586206895 0.7985611510791367
Classified by:  Counter({'counts between': 75, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 7, 'compartment mir gene': 7, 'counts opp': 6, 'counts between equal': 5, 'compartment gene mir': 4, 'compartment mir gene neg corr': 4, 'compartment gene mir by': 3, 'between mir gene reg corr': 2, 'counts between alternating': 1})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[ 80   

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    149 0.6422413793103449
Incorrect:  83 0.3577586206896552
classes Counter()
precision 0.9856115107913669
recall 0.6284403669724771
specificity 0.8571428571428571
f1 0.7675070028011205
Correct classified:      111 0.47844827586206895 0.7985611510791367
Incorrect classified:    28 0.1206896551724138 0.2014388489208633
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 24, 'between mir gene': 2, 'compartment mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      109 0.4698275862068966 0.7841726618705036
Classified by:  Counter({'counts between': 75, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 7, 'compartment mir gene': 7, 'counts opp': 6, 'counts between equal': 5, 'compartment gene mir': 4, 'compartment mir gene neg corr': 4, 'compartment gene mir by': 3, 'between mir gene reg corr': 2, 'counts between alternating': 1})
[[[133   2]
  [  0   4]]

 [[13

...

Total:      397
Correct:    314 0.7909319899244333
Incorrect:  83 0.20906801007556675
classes Counter()
precision 0.845360824742268
recall 0.7557603686635944
specificity 0.8333333333333334
f1 0.7980535279805352
Correct classified:      150 0.3778337531486146 0.7731958762886598
Incorrect classified:    44 0.11083123425692695 0.2268041237113402
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 19, 'compartment mir gene': 12, 'counts between equal': 3, 'between mir gene': 3, 'compartment gene mir': 2, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      126 0.31738035264483627 0.6494845360824743
Classified by:  Counter({'counts between': 103, 'compartment mir gene': 21, 'between mir gene': 19, 'between gene mir': 12, 'counts between equal': 10, 'compartment gene mir by': 6, 'compartment gene mir': 5, 'counts opp': 4, 'compartment mi

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    318 0.801007556675063
Incorrect:  79 0.19899244332493704
classes Counter()
precision 0.8144329896907216
recall 0.7860696517412935
specificity 0.8163265306122449
f1 0.7999999999999999
Correct classified:      150 0.3778337531486146 0.7731958762886598
Incorrect classified:    44 0.11083123425692695 0.2268041237113402
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 19, 'compartment mir gene': 12, 'counts between equal': 3, 'between mir gene': 3, 'compartment gene mir': 2, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      124 0.3123425692695214 0.6391752577319587
Classified by:  Counter({'counts between': 103, 'compartment mir gene': 21, 'between mir gene': 19, 'between gene mir': 12, 'counts between equal': 10, 'compartment gene mir by': 6, 'compartment gene mir': 5, 'counts opp': 4, 'compartment mir

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    318 0.801007556675063
Incorrect:  79 0.19899244332493704
classes Counter()
precision 0.7783505154639175
recall 0.8074866310160428
specificity 0.7952380952380952
f1 0.7926509186351706
Correct classified:      150 0.3778337531486146 0.7731958762886598
Incorrect classified:    44 0.11083123425692695 0.2268041237113402
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 19, 'compartment mir gene': 12, 'counts between equal': 3, 'between mir gene': 3, 'compartment gene mir': 2, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      117 0.2947103274559194 0.6030927835051546
Classified by:  Counter({'counts between': 103, 'compartment mir gene': 21, 'between mir gene': 19, 'between gene mir': 12, 'counts between equal': 10, 'compartment gene mir by': 6, 'compartment gene mir': 5, 'counts opp': 4, 'compartment mir

In [11]:
%aimport MirGeneRelCheck

results_test_scilg_bionlp = runCheck(-1, "TEST", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)
results_train_scilg_bionlp = runCheck(-1, "TRAIN", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 1 subsets.
Will test classification: 1 subsets.
Testing checks ('conj', 'sdp', 'compartment', 'context', 'entity')
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    213 0.9181034482758621
Incorrect:  19 0.08189655172413793
classes Counter()
precision 0.9136690647482014
recall 0.9477611940298507
specificity 0.8775510204081632
f1 0.9304029304029303
Correct classified:      127 0.5474137931034483 0.9136690647482014
Incorrect classified:    12 0.05172413793103448 0.08633093525179857
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      117 0.5043103448275862 0.841726618705036
Classified by:  Counter({'counts between': 64, 'counts opp': 10, 'counts between equal': 10, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'compartment gene mir by': 4, 'compartment mir gene neg corr': 4, 'between mir gene reg corr': 3, 'counts between alternating': 2})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    359 0.9042821158690176
Incorrect:  38 0.09571788413098237
classes Counter()
precision 0.9278350515463918
recall 0.8823529411764706
specificity 0.927461139896373
f1 0.9045226130653266
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      156 0.3929471032745592 0.8041237113402062
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene 

In [12]:
%aimport MirGeneRelCheck

results_test_scilg_scilg = runCheck(-1, "TEST", nlp, nlp, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)
results_train_scilg_scilg = runCheck(-1, "TRAIN", nlp, nlp, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 1 subsets.
Will test classification: 1 subsets.
Testing checks ('conj', 'sdp', 'compartment', 'context', 'entity')
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    213 0.9181034482758621
Incorrect:  19 0.08189655172413793
classes Counter()
precision 0.9136690647482014
recall 0.9477611940298507
specificity 0.8775510204081632
f1 0.9304029304029303
Correct classified:      127 0.5474137931034483 0.9136690647482014
Incorrect classified:    12 0.05172413793103448 0.08633093525179857
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      117 0.5043103448275862 0.841726618705036
Classified by:  Counter({'counts between': 64, 'counts opp': 10, 'counts between equal': 10, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'compartment gene mir by': 4, 'compartment mir gene neg corr': 4, 'between mir gene reg corr': 3, 'counts between alternating': 2})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    361 0.9093198992443325
Incorrect:  36 0.0906801007556675
classes Counter()
precision 0.9381443298969072
recall 0.883495145631068
specificity 0.93717277486911
f1 0.9099999999999999
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      156 0.3929471032745592 0.8041237113402062
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene pos

In [13]:
%aimport MirGeneRelCheck

results_test_bionlp_bionlp = runCheck(-1, "TEST", nlp_ent, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)
results_train_bionlp_bionlp = runCheck(-1, "TRAIN", nlp_ent, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 1 subsets.
Will test classification: 1 subsets.
Testing checks ('conj', 'sdp', 'compartment', 'context', 'entity')
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    183 0.7887931034482759
Incorrect:  49 0.21120689655172414
classes Counter()
precision 0.8057553956834532
recall 0.835820895522388
specificity 0.7244897959183674
f1 0.8205128205128205
Correct classified:      128 0.5517241379310345 0.920863309352518
Incorrect classified:    11 0.04741379310344827 0.07913669064748201
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 10, 'between mir gene': 1})
Correct interaction&classified:      103 0.44396551724137934 0.7410071942446043
Classified by:  Counter({'counts between': 66, 'counts between equal': 10, 'between gene mir': 10, 'counts opp': 9, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 6, 'compartment mir gene': 6, 'compartment gene mir': 5, 'between mir gene reg corr': 5, 'compartment mir gene neg corr': 4, 'compartment gene mir by': 2, 'counts between alternating': 1})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0   1]]

 [[ 81   4]
  [  5  49]]

 [[ 57   3]

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    332 0.836272040302267
Incorrect:  65 0.163727959697733
classes Counter()
precision 0.8814432989690721
recall 0.8028169014084507
specificity 0.875
f1 0.8402948402948403
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      150 0.3778337531486146 0.7731958762886598
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene pos corr': 4, '

In [14]:
%aimport MirGeneRelCheck

results_test_spcylg_bionlp = runCheck(-1, "TEST", nlp_weblg, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)
results_train_spcylg_bionlp = runCheck(-1, "TRAIN", nlp_weblg, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=False)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 1 subsets.
Will test classification: 1 subsets.
Testing checks ('conj', 'sdp', 'compartment', 'context', 'entity')
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    169 0.728448275862069
Incorrect:  63 0.27155172413793105
classes Counter()
precision 0.7266187050359713
recall 0.8015873015873016
specificity 0.6415094339622641
f1 0.7622641509433962
Correct classified:      111 0.47844827586206895 0.7985611510791367
Incorrect classified:    28 0.1206896551724138 0.2014388489208633
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'counts between': 24, 'between mir gene': 2, 'compartment mir gene': 1, 'counts between alternating': 1})
Correct interaction&classified:      87 0.375 0.6258992805755396
Classified by:  Counter({'counts between': 75, 'between gene mir': 10, 'compartment mir gene by': 8, 'return': 7, 'between mir gene': 7, 'compartment mir gene': 7, 'counts opp': 6, 'counts between equal': 5, 'compartment gene mir': 4, 'compartment mir gene neg corr': 4, 'compartment gene mir by': 3, 'between mir gene reg corr': 2, 'counts between alternating': 1})
[[[133   2]
  [  0   4]]

 [[138   0]
  [  0 

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    318 0.801007556675063
Incorrect:  79 0.19899244332493704
classes Counter()
precision 0.7783505154639175
recall 0.8074866310160428
specificity 0.7952380952380952
f1 0.7926509186351706
Correct classified:      150 0.3778337531486146 0.7731958762886598
Incorrect classified:    44 0.11083123425692695 0.2268041237113402
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 19, 'compartment mir gene': 12, 'counts between equal': 3, 'between mir gene': 3, 'compartment gene mir': 2, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      117 0.2947103274559194 0.6030927835051546
Classified by:  Counter({'counts between': 103, 'compartment mir gene': 21, 'between mir gene': 19, 'between gene mir': 12, 'counts between equal': 10, 'compartment gene mir by': 6, 'compartment gene mir': 5, 'counts opp': 4, 'compartment mir

In [15]:
%aimport MirGeneRelCheck

results_test_scilg_bionlp_subsetc = runCheck(-1, "TEST", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=True)
results_train_scilg_bionlp_subsetc = runCheck(-1, "TRAIN", nlp, nlp_ent, relationsFile=relationsFile, subset_interactions=False, subset_classification=True)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test interaction: 1 subsets.
Will test classification: 16 subsets.
Testing checks ('conj', 'sdp', 'compartment', 'context', 'entity')
Testing checks ()


Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      232
Correct:    213 0.9181034482758621
Incorrect:  19 0.08189655172413793
classes Counter()
precision 0.9136690647482014
recall 0.9477611940298507
specificity 0.8775510204081632
f1 0.9304029304029303
Correct classified:      64 0.27586206896551724 0.460431654676259
Incorrect classified:    75 0.3232758620689655 0.539568345323741
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'static': 75})
Correct interaction&classified:      53 0.22844827586206898 0.381294964028777
Classified by:  Counter({'static': 129, 'between gene mir': 10})
[[[135   0]
  [  4   0]]

 [[138   0]
  [  1   0]]

 [[ 10  75]
  [  0  54]]

 [[ 60   0]
  [ 69  10]]

 [[138   0]
  [  1   0]]]
               precision    recall  f1-score   support

 GENE_MIR_NEU       0.00      0.00      0.00         4
  GENE_MIR_UP       0.00      0.00      0.00         1
MIR_GENE_DOWN       0.42      1.00      0.59        54
 MIR_GENE_NEU       1.00      0.13      0.22        79
  MIR_GENE_UP       0

/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/pyth

Total:      232
Correct:    213 0.9181034482758621
Incorrect:  19 0.08189655172413793
classes Counter()
precision 0.9136690647482014
recall 0.9477611940298507
specificity 0.8775510204081632
f1 0.9304029304029303
Correct classified:      87 0.375 0.6258992805755396
Incorrect classified:    52 0.22413793103448276 0.37410071942446044
Random classified:  54 0.23275862068965517 0.38848920863309355
Counter({'static': 52})
Correct interaction&classified:      76 0.3275862068965517 0.5467625899280576
Classified by:  Counter({'static': 102, 'between gene mir': 10, 'compartment mir gene by': 8, 'compartment mir gene': 6, 'compartment gene mir': 5, 'compartment gene mir by': 4, 'compartment mir gene neg corr': 4})
[[[135   0]
  [  4   0]]

 [[138   0]
  [  1   0]]

 [[ 33  52]
  [  0  54]]

 [[ 60   0]
  [ 46  33]]

 [[138   0]
  [  1   0]]]
               precision    recall  f1-score   support

 GENE_MIR_NEU       0.00      0.00      0.00         4
  GENE_MIR_UP       0.00      0.00      0.00  

...

Total:      397
Correct:    359 0.9042821158690176
Incorrect:  38 0.09571788413098237
classes Counter()
precision 0.9278350515463918
recall 0.8823529411764706
specificity 0.927461139896373
f1 0.9045226130653266
Correct classified:      160 0.40302267002518893 0.8247422680412371
Incorrect classified:    34 0.08564231738035265 0.17525773195876287
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 17, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'return': 2, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      152 0.38287153652392947 0.7835051546391752
Classified by:  Counter({'counts between': 118, 'counts between equal': 17, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'return': 6, 'compartment mir gene pos corr': 4, 'compartment m

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    359 0.9042821158690176
Incorrect:  38 0.09571788413098237
classes Counter()
precision 0.9278350515463918
recall 0.8823529411764706
specificity 0.927461139896373
f1 0.9045226130653266
Correct classified:      170 0.4282115869017632 0.8762886597938144
Incorrect classified:    24 0.060453400503778336 0.12371134020618557
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 14, 'counts between equal': 4, 'counts opp': 3, 'between mir gene reg corr': 1, 'between mir gene rb': 1, 'between mir gene neg corr': 1})
Correct interaction&classified:      160 0.40302267002518893 0.8247422680412371
Classified by:  Counter({'counts between': 106, 'between mir gene': 25, 'between gene mir': 19, 'counts between equal': 16, 'counts opp': 9, 'return': 6, 'between mir gene pos corr': 3, 'between mir gene reg corr': 3, 'between mir gene neg corr': 3, 'between gene mir sites': 2, 'between mir gene rb': 1, 'counts between alternating': 1})
[[[18

Setting relfile /mnt/extproj/projekte/textmining//obodir/allrels.csv


Total:      397
Correct:    359 0.9042821158690176
Incorrect:  38 0.09571788413098237
classes Counter()
precision 0.9278350515463918
recall 0.8823529411764706
specificity 0.927461139896373
f1 0.9045226130653266
Correct classified:      165 0.4156171284634761 0.8505154639175257
Incorrect classified:    29 0.07304785894206549 0.14948453608247422
Random classified:  53 0.13350125944584382 0.27319587628865977
Counter({'counts between': 13, 'counts between equal': 4, 'compartment gene mir': 3, 'counts opp': 3, 'compartment mir gene by': 2, 'compartment mir gene pos corr': 1, 'between mir gene reg corr': 1, 'compartment mir gene': 1, 'compartment mir gene neg corr': 1})
Correct interaction&classified:      156 0.3929471032745592 0.8041237113402062
Classified by:  Counter({'counts between': 101, 'counts between equal': 16, 'between mir gene': 16, 'between gene mir': 11, 'compartment mir gene': 10, 'counts opp': 9, 'compartment gene mir': 7, 'compartment gene mir by': 6, 'compartment mir gene 

In [16]:
outDirectory = mx_base + "/scai_eval/"

In [17]:
if not os.path.isdir(outDirectory):
    os.makedirs(outDirectory, exist_ok=True)

In [18]:
with open(outDirectory+"/results_test_scilg_bionlp.pickle", "wb") as fout:
    pickle.dump(results_test_scilg_bionlp, fout)   

with open(outDirectory+"/results_train_scilg_bionlp.pickle", "wb") as fout:
    pickle.dump(results_train_scilg_bionlp, fout)

In [19]:
with open(outDirectory+"/results_test_scilg_scilg.pickle", "wb") as fout:
    pickle.dump(results_test_scilg_scilg, fout)   

with open(outDirectory+"/results_train_scilg_scilg.pickle", "wb") as fout:
    pickle.dump(results_train_scilg_scilg, fout)

In [20]:
with open(outDirectory+"/results_test_bionlp_bionlp.pickle", "wb") as fout:
    pickle.dump(results_test_bionlp_bionlp, fout)   

with open(outDirectory+"/results_train_bionlp_bionlp.pickle", "wb") as fout:
    pickle.dump(results_train_bionlp_bionlp, fout)

In [21]:
with open(outDirectory+"/results_test_spcylg_bionlp.pickle", "wb") as fout:
    pickle.dump(results_test_spcylg_bionlp, fout)   

with open(outDirectory+"/results_train_spcylg_bionlp.pickle", "wb") as fout:
    pickle.dump(results_train_spcylg_bionlp, fout)

In [22]:
[x for x in results_test_spcylg_bionlp]

[(('conj', 'sdp', 'compartment', 'context', 'entity'),
  ('compartment', 'between', 'counts', 'return'))]

In [23]:
! ls ../scai_eval

ls: cannot access '../scai_eval': No such file or directory


In [24]:
with open(outDirectory+"/results_test_scilg_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_test_scilg_bionlp_subseti, fout)   

with open(outDirectory+"/results_train_scilg_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_train_scilg_bionlp_subseti, fout)

In [25]:
with open(outDirectory+"/results_test_scilg_bionlp_subsetc.pickle", "wb") as fout:
    pickle.dump(results_test_scilg_bionlp_subsetc, fout)   

with open(outDirectory+"/results_train_scilg_bionlp_subsetc.pickle", "wb") as fout:
    pickle.dump(results_train_scilg_bionlp_subsetc, fout)

In [26]:
with open(outDirectory+"/results_test_spcylg_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_test_spcylg_bionlp_subseti, fout)   

with open(outDirectory+"/results_train_spcylg_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_train_spcylg_bionlp_subseti, fout)

In [27]:
with open(outDirectory+"/results_test_bionlp_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_test_bionlp_bionlp_subseti, fout)   

with open(outDirectory+"/results_train_bionlp_bionlp_subseti.pickle", "wb") as fout:
    pickle.dump(results_train_bionlp_bionlp_subseti, fout)